In [1]:
import pandas as pd

playlist_data = pd.read_csv('./spotify_data/playlistCSV.csv', header=0)
streaming_history = pd.read_csv('./spotify_data/streamingHistoryCSV.csv', header=0)
playlist_data.head()

,albumName,artistName,trackName,trackUri,playlists.name
0,...Like Clockwork,Queens of the Stone Age,I Sat by the Ocean,spotify:track:1D3YGmST01Ba0WRiT7THPG,The Knew
1,¡DOS!,Green Day,Stray Heart,spotify:track:2Q22sa3Y4lKXHHBBAxEhYY,Missing Her
2,¡DOS!,Green Day,Stray Heart,spotify:track:2Q22sa3Y4lKXHHBBAxEhYY,Perfect Playlist Pt 2
3,¡UNO!,Green Day,Nuclear Family,spotify:track:206n37mZTmloL45RYhmMlP,Missing Her
4,¡UNO!,Green Day,Nuclear Family,spotify:track:206n37mZTmloL45RYhmMlP,Rock Paper Scissors


In [2]:
merged_dataset = streaming_history.merge(playlist_data, on='trackName',)
merged_dataset['time'] = merged_dataset['time'].str.split(' ').str[1]
merged_dataset['track_id'] = merged_dataset['trackUri'].str.split('track:').str[1]


In [3]:
def month_to_int(value):
    if value == 'June':
        return 6
    elif value == 'July':
        return 7
    elif value == 'August':
        return 8
    else:
        return 9

In [4]:
merged_dataset['Month'] = merged_dataset['Month'].apply(month_to_int)

In [7]:
merged_dataset['date'] = merged_dataset['Month'].astype(str) + '-' + merged_dataset['Day'].astype(str) + '-' + merged_dataset['Year'].astype(str)
date_format = '%m-%d-%Y'
merged_dataset['date'] = pd.to_datetime(merged_dataset['date'], format=date_format)
merged_dataset['hour'] = merged_dataset['time'].str.split(':').str[0]
#merged_dataset.to_csv('./cleaned_data/mergedSpotifyData.csv', index=False)

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

client_id = '3811b236a5dd410e9f8bb2321bbdef9f'
client_secret = '5daa963dee6d41dd820c8d2a776c1fcc'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
requests_per_minute = 150
requests_per_hour = 9000

for index, track_uri in enumerate(merged_dataset['trackUri']):
    audio_features = sp.audio_features(track_uri)
    api_dataset = pd.DataFrame(audio_features)
    print(audio_features)

    time.sleep(30)

    if index % requests_per_minute == 0 and index != 0:
        time_to_sleep = 60
        time.sleep(time_to_sleep)
        index = 0

    if index % requests_per_hour == 0 and index != 0:
        time_to_sleep = 3600
        time.sleep(time_to_sleep)
        index = 0

[{'danceability': 0.728, 'energy': 0.448, 'key': 8, 'loudness': -10.54, 'mode': 1, 'speechiness': 0.172, 'acousticness': 0.103, 'instrumentalness': 0, 'liveness': 0.267, 'valence': 0.483, 'tempo': 80.902, 'type': 'audio_features', 'id': '5r96TaQquRrlo3Ym3ZlSL2', 'uri': 'spotify:track:5r96TaQquRrlo3Ym3ZlSL2', 'track_href': 'https://api.spotify.com/v1/tracks/5r96TaQquRrlo3Ym3ZlSL2', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5r96TaQquRrlo3Ym3ZlSL2', 'duration_ms': 202920, 'time_signature': 4}]
[{'danceability': 0.728, 'energy': 0.448, 'key': 8, 'loudness': -10.54, 'mode': 1, 'speechiness': 0.172, 'acousticness': 0.103, 'instrumentalness': 0, 'liveness': 0.267, 'valence': 0.483, 'tempo': 80.902, 'type': 'audio_features', 'id': '5r96TaQquRrlo3Ym3ZlSL2', 'uri': 'spotify:track:5r96TaQquRrlo3Ym3ZlSL2', 'track_href': 'https://api.spotify.com/v1/tracks/5r96TaQquRrlo3Ym3ZlSL2', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5r96TaQquRrlo3Ym3ZlSL2', 'duration_ms': 2029

KeyboardInterrupt: 

In [10]:
columns_to_drop = ['analysis_url','track_href','type','uri']
api_dataset.drop(columns_to_drop, axis=1,inplace=True)
api_dataset.rename(columns={'id': 'track_id'}, inplace=True)


In [12]:
spotify_dataset = merged_dataset.merge(api_dataset, on='track_id')
columns_to_drop = ['trackUri','Sum of msPlayed','artistName_x']
spotify_dataset.drop(columns_to_drop, axis=1,inplace=True)
spotify_dataset.rename(columns={'artistName_y': 'artistName'}, inplace=True)
spotify_dataset.to_csv('apiData.csv', index=False)